In [2]:
cd app

/home/ubuntu/ml-law/eula-scan/app


In [3]:
from app import model
from app import helpers # gotta fix the import of model -> '.model' in helpers.py, but that breaks the app.

In [4]:
import requests
import datetime
import json
import time

In [5]:
company_row = model.lookup_company_metadata(143)

In [6]:
company_row

{'id': 143,
 'url': 'http://foursquare.com/legal/privacy',
 'name': 'PP v7 data - Foursquare',
 'last_scan': 1576444710,
 'settings': '{"filter_start": "", "filter_end": ""}',
 'status': None}

In [7]:
def clear():
    model._ex("delete from tos_text where start_date < 1500000000")


In [8]:
def _enumerate_company_archive(company_row):
    query_string = {
        "url": company_row['url'],
        "limit": 1000,
        "showResumeKey": "true",
        "output": "json",
        "resumeKey": None,
    }
    all_rows = []
    while True:
        r = requests.get("http://web.archive.org/cdx/search/cdx", params=query_string)
        output = r.json()
        keys = output[0]
        end_idx = None
        if output[-2] == []:
            query_string['resumeKey'] = output[-1][0]
            end_idx = -2
        all_rows.extend(dict(zip(keys, oo)) for oo in output[1:end_idx])
        if output[-2] != []:
            break
    return all_rows

def _add_archived_TOS(company_row, archive_object):
    timestamp = int(datetime.datetime.strptime(archive_object['timestamp'], "%Y%m%d%H%M%S").timestamp())
    print(archive_object['timestamp'])
    next_pull = model.lookup_next_TOS(company_row['id'], timestamp)
    prev_pull = model.lookup_TOS(company_row['id'], timestamp)
    if next_pull['timestamp'] == timestamp:
        print("duplicate timestamp")
    current_pull = helpers._pull_TOS("http://web.archive.org/web/{timestamp}/{original}".format(**archive_object))
    unchanged_next = next_pull and helpers._diff_test(current_pull['text'], next_pull['text'], json.loads(company_row['settings']))
    unchanged_prev = prev_pull and helpers._diff_test(current_pull['text'], prev_pull['text'], json.loads(company_row['settings']))
    if unchanged_next:
        print("earlier version of already loaded")
        model.backdate_TOS(next_pull['tos_id'], timestamp)
    elif unchanged_prev:
        print("no change since last pull")
    else:
        print("changed")
        model.add_TOS(company_row["id"], current_pull['text'], timestamp)


def pull_company_archive(company_id, start_point=None):
    company_row = model.lookup_company_metadata(company_id)
    archive_rows = _enumerate_company_archive(company_row)
    for archive_row in archive_rows:
        if start_point and archive_row['timestamp'] < start_point:
            continue
        try:
            _add_archived_TOS(company_row, archive_row)
        except:
            time.sleep(10)
            print("EXCEPTION")
            _add_archived_TOS(company_row, archive_row)
            print("TRUE FAILURE")

In [8]:
company_row = model.lookup_company_metadata(8)
archive_rows = _enumerate_company_archive(company_row)

In [9]:
for archive_row in archive_rows:
    if archive_row['timestamp'] < '20191118023930':
        continue
    try:
        _add_archived_TOS(company_row, archive_row)
    except:
        time.sleep(10)
        print("EXCEPTION")
        _add_archived_TOS(company_row, archive_row)
        print("TRUE FAILURE")

20160320171951
no change since last pull
20160321175440
no change since last pull
20160322092000
changed
20160322182948
changed
20160323095550
no change since last pull
20160323192342
no change since last pull
20160324200833
no change since last pull
20160325203936
no change since last pull
20160326210819
no change since last pull
20160327214043
no change since last pull
20160328223210
no change since last pull
20160329230251
changed
20160330081424
no change since last pull
20160331000059
no change since last pull
20160401002450
no change since last pull
20160402005555
no change since last pull
20160403013219
no change since last pull
20160404015720
no change since last pull
20160405023752
no change since last pull
20160406045704
no change since last pull
20160406081800
no change since last pull
20160407064856
no change since last pull
20160407145941
no change since last pull
20160408055553
no change since last pull
20160409062722
no change since last pull
20160410071320
no change sinc

no change since last pull
20161112034129
no change since last pull
20161113063153
no change since last pull
20161113133213
no change since last pull
20161113212043
no change since last pull
20161114064438
no change since last pull
20161115074749
no change since last pull
20161115143833
no change since last pull
20161118220931
changed
20161119040556
no change since last pull
20161121043320
no change since last pull
20161122065019
EXCEPTION
20161122065019
no change since last pull
TRUE FAILURE
20161122133317
no change since last pull
20161124081530
no change since last pull
20161124144741
no change since last pull
20161125084232
no change since last pull
20161125151009
no change since last pull
20161126084835
no change since last pull
20161127070815
no change since last pull
20161128091836
no change since last pull
20161128162229
no change since last pull
20161129094501
changed
20161129161142
no change since last pull
20161130081248
no change since last pull
20161201102747
no change sinc

no change since last pull
20170505044505
no change since last pull
20170506052739
no change since last pull
20170507050752
no change since last pull
20170507094233
no change since last pull
20170508061228
no change since last pull
20170509062449
no change since last pull
20170510062845
no change since last pull
20170510130341
no change since last pull
20170511065843
no change since last pull
20170512013753
no change since last pull
20170512075347
no change since last pull
20170513053500
changed
20170513085205
changed
20170514023846
changed
20170514023847
no change since last pull
20170514023848
no change since last pull
20170514092624
changed
20170515095126
no change since last pull
20170516094611
no change since last pull
20170516142908
no change since last pull
20170517103607
no change since last pull
20170518111052
no change since last pull
20170519114745
no change since last pull
20170520113559
no change since last pull
20170520180121
no change since last pull
20170521114001
no cha

no change since last pull
20170918152018
no change since last pull
20170918182440
no change since last pull
20170919154514
no change since last pull
20170919185444
no change since last pull
20170920161726
no change since last pull
20170920192523
no change since last pull
20170923173257
no change since last pull
20170925093759
no change since last pull
20170930153157
changed
20171002200120
no change since last pull
20171003231852
changed
20171004020832
no change since last pull
20171004234940
no change since last pull
20171005024008
no change since last pull
20171005205602
no change since last pull
20171006004824
no change since last pull
20171006033644
no change since last pull
20171006171938
changed
20171007020446
changed
20171007050251
no change since last pull
20171008023222
no change since last pull
20171008051909
no change since last pull
20171009032027
no change since last pull
20171009060745
no change since last pull
20171010042747
no change since last pull
20171010071826
no cha

no change since last pull
20180211124459
no change since last pull
20180212102428
no change since last pull
20180212132002
no change since last pull
20180213115336
no change since last pull
20180213151103
no change since last pull
20180214122207
no change since last pull
20180214152222
changed
20180214182023
no change since last pull
20180215131716
changed
20180215162033
no change since last pull
20180216135026
no change since last pull
20180216165959
no change since last pull
20180217144108
no change since last pull
20180217174849
no change since last pull
20180218144739
no change since last pull
20180218174847
no change since last pull
20180219154149
no change since last pull
20180219184453
no change since last pull
20180220160912
no change since last pull
20180220193037
no change since last pull
20180221003304
no change since last pull
20180221170839
no change since last pull
20180221201428
no change since last pull
20180222174349
no change since last pull
20180222210238
no change s

no change since last pull
20180524120450
no change since last pull
20180525121005
no change since last pull
20180526054230
changed
20180526123807
changed
20180526150703
no change since last pull
20180527125532
no change since last pull
20180528132433
no change since last pull
20180528195348
no change since last pull
20180529142227
no change since last pull
20180529221403
no change since last pull
20180530143137
no change since last pull
20180530224232
no change since last pull
20180531144439
no change since last pull
20180531171655
no change since last pull
20180601150654
changed
20180602074054
changed
20180602154455
changed
20180603160728
no change since last pull
20180604165253
no change since last pull
20180605004604
no change since last pull
20180605142252
no change since last pull
20180605151651
no change since last pull
20180606052731
no change since last pull
20180607063313
changed
20180608065422
no change since last pull
20180609070516
no change since last pull
20180610080223
n

changed
20181101181757
no change since last pull
20181101203805
no change since last pull
20181102001542
changed
20181103003947
no change since last pull
20181103034409
no change since last pull
20181104011630
no change since last pull
20181105015928
no change since last pull
20181106030254
no change since last pull
20181106035316
changed
20181107034428
changed
20181107112256
changed
20181108051248
changed
20181108171615
changed
20181108213223
no change since last pull
20181109091433
changed
20181109115957
changed
20181110100915
changed
20181110193904
changed
20181110220558
no change since last pull
20181111103513
changed
20181111211120
changed
20181112115103
changed
20181113123007
no change since last pull
20181114132930
no change since last pull
20181114152047
changed
20181114181141
no change since last pull
20181115014109
changed
20181115081828
no change since last pull
20181115132308
changed
20181116134259
no change since last pull
20181116194631
changed
20181117142801
changed
2018

changed
20190319214145
no change since last pull
20190320032929
changed
20190320220938
changed
20190321224709
no change since last pull
20190322034832
changed
20190322233934
changed
20190324003054
no change since last pull
20190324073658
changed
20190324144408
no change since last pull
20190325004945
changed
20190326011426
no change since last pull
20190327020559
no change since last pull
20190328022418
no change since last pull
20190329035144
no change since last pull
20190329182451
changed
20190329221448
no change since last pull
20190330031718
no change since last pull
20190330033446
no change since last pull
20190330041023
changed
20190330183842
no change since last pull
20190331042008
no change since last pull
20190331061932
changed
20190401094513
changed
20190401111436
changed
20190402100251
changed
20190402225318
changed
20190402225318
duplicate timestamp
earlier version of already loaded
20190402225318
duplicate timestamp
earlier version of already loaded
20190402225338
no chan

no change since last pull
20190831175900
no change since last pull
20190901181524
no change since last pull
20190902183634
no change since last pull
20190903185443
no change since last pull
20190904134701
changed
20190904141239
no change since last pull
20190904193300
changed
20190905202543
no change since last pull
20190906202827
no change since last pull
20190907205931
no change since last pull
20190908211910
no change since last pull
20190909215557
no change since last pull
20190910223206
no change since last pull
20190911230256
no change since last pull
20190912234542
no change since last pull
20190913235232
no change since last pull
20190915001900
no change since last pull
20190917055029
no change since last pull
20190918064512
no change since last pull
20190919081414
no change since last pull
20190919231327
no change since last pull
20190920075455
no change since last pull
20190921011542
changed
20190921090154
changed
20190922093019
no change since last pull
20190923103237
no cha

no change since last pull
20191118112219
changed
20191118112219
duplicate timestamp
earlier version of already loaded
20191119071639
no change since last pull
20191119071639
no change since last pull
20191120100721
no change since last pull
20191120100721
no change since last pull
20191121011405
changed
20191121064643
no change since last pull
20191121094948
changed
20191121094948
duplicate timestamp
earlier version of already loaded
20191122083636
changed
20191122124615
changed
20191122124615
duplicate timestamp
earlier version of already loaded
20191123130946
no change since last pull
20191123130946
no change since last pull
20191123163816
changed
20191123163816
duplicate timestamp
earlier version of already loaded
20191124104345
changed
20191124104345
duplicate timestamp
earlier version of already loaded
20191125112727
no change since last pull
20191125112727
no change since last pull
20191126114758
no change since last pull
20191126114758
no change since last pull
20191127035851
ch

In [15]:
pull_company_archive(7, '20191116120019')

20191116120019
EXCEPTION
20191116120019


TypeError: 'NoneType' object is not subscriptable

In [20]:
model._local_ex("select * from company").fetchall()

[(1, 'http://0.0.0.0:8999/www', 'STEIN TEST LOCAL', None, 1576385009, None, "In attempting to download the link, got the following error \n<class 'requests.exceptions.ConnectionError'>: HTTPConnectionPool(host='0.0.0.0', port= ... (58 characters truncated) ... onnectionError('<urllib3.connection.HTTPConnection object at 0x7fbbe01c0f60>: Failed to establish a new connection: [Errno 111] Connection refused'))"),
 (2, 'http://dropbox.com/terms', 'Dropbox', 1575173241.6911333, 1576385010, '{"filter_start": "Dropbox Terms of Service", "filter_end": "privacy or infringe the rights of others."}', "In attempting to download the link, got the following error \n<class 'requests.exceptions.ConnectionError'>: HTTPConnectionPool(host='dropbox.com', p ... (79 characters truncated) ... urllib3.connection.HTTPConnection object at 0x7fbbe0069048>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))"),
 (3, 'http://0.0.0.0:8999/www/index.html', 'Test 2, truncated ter

In [21]:
res = _

In [22]:
res[1]

(2, 'http://dropbox.com/terms', 'Dropbox', 1575173241.6911333, 1576385010, '{"filter_start": "Dropbox Terms of Service", "filter_end": "privacy or infringe the rights of others."}', "In attempting to download the link, got the following error \n<class 'requests.exceptions.ConnectionError'>: HTTPConnectionPool(host='dropbox.com', p ... (79 characters truncated) ... urllib3.connection.HTTPConnection object at 0x7fbbe0069048>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))")

In [28]:
pp7 = [r[:3] for r in res if r[2].startswith('PP')]

In [29]:
for id, url, name in pp7:
    model.create_company(name, url, {})

In [30]:
#helpers.scan_company_tos(id)

In [42]:
for r in model._ex("select * from company where last_scan > 0"):
    if 'PP' in r.name:
        pull_company_archive,(r.id,)).start()

Exception in thread Thread-167:
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-pack

20071103180237
20070912062645
20080519225605
20100623012646
2009052123514720110924200754
20070829104926
20070826154215
20080309074524

20080817020827
20090120072623
20080305174838
20090129070805
2016111009395320150528031553

20070907020915

Exception in thread Thread-277:
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-pack


20170406142521
2011070422391220140714013351

20110720123525
20010714165638
20150108185617
20160725235429
20091205182035
20141127083700
20070609131554


Exception in thread Thread-264:
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/ubuntu/.local/share/virtualenvs/ml-law-cKtHyQ1o/lib/python3.7/site-pack

...

Exception in thread Thread-303:
Traceback (most recent call last):
  File "<ipython-input-12-1253d3de880f>", line 8, in pull_company_archive
    _add_archived_TOS(company_row, archive_row)
  File "<ipython-input-7-1923d51b48a7>", line 8, in _add_archived_TOS
    current_pull = helpers._pull_TOS("http://web.archive.org/web/{timestamp}/{original}".format(**archive_object))
  File "/home/ubuntu/ml-law/eula-scan/app/helpers.py", line 135, in _pull_TOS
    .replace("<BR>", "~~~")
AssertionError: error, code: 503

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-1253d3de880f>", line 12, in pull_company_archive
    _add_archived_TOS(company_row, archive_row)
  File "<ipython-input-7-1923d51b48a7>", line 8, in _add_archived_T

no change since last pull
20130514202852
changed
20121215051842
no change since last pull
20180710053738
no change since last pull
20181203025402
changed
20080822034221
no change since last pull
20130130035954
changed
20130115103657
no change since last pull
20090114005333
no change since last pull
20150623015705
no change since last pull
20181206213946
no change since last pull
20120507003230
no change since last pull
20130807042709
changed
20130115175120
no change since last pull
20180802060719
no change since last pull
20120702124142
changed
20080823081428
no change since last pull
20090224203841
no change since last pull
20181208000945
no change since last pull
20150624065336
no change since last pull
20120507024657
no change since last pull
20130809021516
no change since last pull
20130202015608
no change since last pull
20120805071901
no change since last pull
20181212131446
changedchanged

20130211101321
20090303203741
no change since last pull
20180810062429
no change since las

no change since last pull
20090514072542
no change since last pull
20190315165621
changed
20131016161944
no change since last pull
20160212022413
no change since last pull
20190201034955
no change since last pullchanged

20121014153206
no change since last pull
20140424075351
20171022081455
no change since last pull
20130326010257
no change since last pull
20080910022257
no change since last pull
20120508050134
no change since last pull
20190201180254
no change since last pull
20190402122300
changed
changed
20131118040534
20171122175127
no change since last pull
20160212022859
no change since last pullno change since last pull
20140501191458

20121101065619
no change since last pull
20190204035159
no change since last pull
20190408200203
no change since last pull
20090514072542
no change since last pull
20190208130556
no change since last pull
20121129043653
no change since last pull
20140508193814
changedno change since last pull

20090520082645
no change since last pull
2016032318111

Exception in thread Thread-175:
Traceback (most recent call last):
  File "<ipython-input-12-1253d3de880f>", line 8, in pull_company_archive
    _add_archived_TOS(company_row, archive_row)
  File "<ipython-input-7-1923d51b48a7>", line 8, in _add_archived_TOS
    current_pull = helpers._pull_TOS("http://web.archive.org/web/{timestamp}/{original}".format(**archive_object))
  File "/home/ubuntu/ml-law/eula-scan/app/helpers.py", line 135, in _pull_TOS
    .replace("<BR>", "~~~")
AssertionError: error, code: 503

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-1253d3de880f>", line 12, in pull_company_archive
    _add_archived_TOS(company_row, archive_row)
  File "<ipython-input-7-1923d51b48a7>", line 8, in _add_archived_T

no change since last pull
20120909235707
no change since last pull
20140711033504
changedno change since last pull

20130331005115
no change since last pull
20180604055129
20151119013154
no change since last pull
20160902130834
no change since last pull
20120919221826
changed
no change since last pull
20190310070529
20090420023826
changed
20090723233630
duplicate timestamp
earlier version of already loaded
20090730202452
no change since last pull
20140718083743
no change since last pullchanged
20190310080611

changed
no change since last pull
20090730202452
no change since last pull20090422155545

20160906112313
20121110204607
no change since last pull
20180616132921
no change since last pullno change since last pull
20140725083511

20090805003236
changed
20151229081945
no change since last pull
20160906114936
no change since last pull
20130331122133
no change since last pull
20190311214325
changed
20090505212415
no change since last pull
20180617102321
no change since last pull
201408

In [39]:
import threading

In [40]:
threading.Thread(target=pull_company_archive, args=(r.id)).start()

In [15]:
model._ex("delete from tos_text where id=2565").fetchall()

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [17]:
list(zip(range(5),range(4), range(3)))

[(0, 0, 0), (1, 1, 1), (2, 2, 2)]

In [19]:
import datetime

In [27]:
td = datetime.timedelta(seconds=900)

In [28]:
str(td)

'0:15:00'